In [ ]:
# Instance segmentation: https://youtu.be/lOZDTDOlqfk

from simple_unet_model import simple_unet_model 
from keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from skimage import measure, color, io


IMG_HEIGHT = 256
IMG_WIDTH  = 256
IMG_CHANNELS = 1

def get_model():
    return simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

# load the model and corresponding weights

model = get_model()
model.load_weights('file.hdf5')

In [ ]:
# load and process the test image - image that needs to be segmented. 
test_img = cv2.imread('C:/Users/.tiff', 0)
test_img = Image.fromarray(test_img)
test_img = test_img.resize((IMG_HEIGHT, IMG_WIDTH))
test_img_norm = np.expand_dims(normalize(np.array(test_img), axis=1),2)
test_img_norm=test_img_norm[:,:,0][:,:,None]
test_img_input=np.expand_dims(test_img_norm, 0)

# predict and threshold for values above 0.7 probability
segmented = (model.predict(test_img_input)[0,:,:,0] > 0.7).astype(np.uint8)

# print testing and segmented image
plt.figure(figsize=(8, 8))
plt.subplot(221)
plt.title('Testing Image')
plt.imshow(test_img, cmap='gray')
plt.subplot(222)
plt.title('Segmented Image')
plt.imshow(segmented, cmap='gray')
plt.show()

# save segmented image
plt.imsave('C:/Users/.jpg', segmented, cmap='gray')

In [ ]:
# transform unet image to binary image
img = cv2.imread('C:/Users/.jpg')  
img_grey = img[:,:,0]

# threshold image to binary using OTSU. ALl thresholded pixels will be set to 255.
ret1, thresh = cv2.threshold(img_grey, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# identify background pixels using watershed 
sure_bg = cv2.dilate(opening,kernel,iterations=10)

# identify foreground pixels using watershed
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)

# threshold the distance transform 
ret2, sure_fg = cv2.threshold(dist_transform, 0.2*dist_transform.max(),255,0)

# identify unsure region as background - foreground
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

# create a marker and label the regions inside using ConnectedComponents
# For sure regions, both foreground and background will be labeled with positive numbers
# Unknown regions will be labeled 0
ret3, markers = cv2.connectedComponents(sure_fg)

# rightnow the entire background pixels is given value 0
# add 10 to all labels so that sure background is not 0, but 10
markers = markers+10

# mow, mark the region of unknown with zero
markers[unknown==255] = 0

# watershed filling
markers = cv2.watershed(img, markers)

# color boundaries in yellow. 
img[markers == -1] = [0,255,255]  

img2 = color.label2rgb(markers, bg_label=0)

In [ ]:
# extract properties of the image 
props = measure.regionprops_table(markers, intensity_image=img_grey, 
                              properties=['area',
                                          'mean_intensity'])
    
import pandas as pd
df = pd.DataFrame(props)

# remove background or other regions that may be counted as objects
df = df[df.mean_intensity > 100] 

In [ ]:
# properties of the image (area)
print(df)

# number of lamellopodia according to area analysis
area = df["area"].sum()
mean = df.loc[df["area"] < 1000, ['area']].mean()
number = area / mean
print(number)